In [11]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [12]:
import cv2
from cv2 import aruco
from matplotlib import pyplot as plt
import math
from math import atan2, cos, sin, sqrt, pi
import numpy as np
from sympy import Segment, Point, Ray
import time

In [13]:
set_var(motor_left_target = 60)
set_var(motor_right_target = 60)

In [14]:
set_var(motor_left_target = 0)
set_var(motor_right_target = 0)

In [15]:
def convertWheelSpeeds(speed_l,speed_r,speed_factor=3,axle_track=94):
    '''
    Function that takes the raw values of each motor angular speed and converts it into
    translation speed (in mm/s) and rotation speed (rad/sec) with a pixel scaling factor.

    Inputs: - speed_l: speed of left motor (no units, from Thymio measurements)
            - speed_r: speed of right motor (no units, from Thymio measurements)
            - speed_factor: 3 (motor_target of 200 sent to Thymio corresponds to a wheel speed of 66.8 mm/s)
            - axle_track: 94mm (distance between both wheels)

    Output: - v  : The current forward velocity of the Thymio (in mm/s)
            - omega : The current angular velocity of the Thymio (in rad/s)(positive direction corresponding to that of the camera frame)
    '''
    # wheel_radius : 22mm
    # axle track : 94mm (distance between the 2 wheels)
    v = 0.5*(speed_l + speed_r) / speed_factor
    omega = (speed_l - speed_r) / (0.5*axle_track*speed_factor)
    
    return v, omega
    

In [16]:
def px2real(x_px, scalingFactor):
    '''
    Function that converts measurement in pixels to measurement in mm.

    Inputs: - x_px : value in pixels
            - scalingFactor : for image size (1920x1080) corresponding to real length and width (800mm x 450mm), the scalingFactor is 2.4 

    Output: - x_real : value in mm
            
    '''
    x_real = x_px / 2.4
    return x_real
    
    

In [17]:
def real2px(x_real, scalingFactor):
    '''
    Function that converts measurement in mm to measurement in pixels.

    Inputs: - x_real : value in mm
            - scalingFactor : for image size (1920x1080) corresponding to real length and width (800mm x 450mm), the scalingFactor is 2.4 

    Output: - x_px : value in pixels
            
    '''
    x_px = x_real * 2.4
    return x_px
    

In [18]:
'''
Extended Kalman Filter class

Code modified from https://github.com/L42Project/Tutoriels/blob/master/Divers/tutoriel36/KalmanFilter.py
Comments describing Kalman Filter taken from https://stackoverflow.com/questions/74318200/how-to-tune-extended-kalman-filter-on-pykalman

Example of instance: KF=KalmanFilter(0.03333, [0, 0, 0, 0, 0]) (dt = 0.033 is the time between each measurement, x_ini = [0, 0, 0, 0, 0] is the initial state)

For update() function : takes as input the measurement vector z
z[0:2] are the measurements from the camera (px, py, orientation)
z[3:4] are the measurements from the Thymio (forward speed v and angular speed omega)

When Thymio position not found by camera -> z becomes 2x1 vector, with z[0:1] being the forward speed v and angular speed omega
'''


class KalmanFilter():
    def __init__(self, dt, x_ini):
        self.dt = dt

        # Initial State Vector
        self.x=np.matrix([[x_ini[0]], [x_ini[1]], [x_ini[2]], [x_ini[3]], [x_ini[4]]])
    

        # Initialize State Transition Matrix (Warning : changes with time -> updateFk())
        self.Fk=np.eye(5)

        # Initialize Measurement matrix H
        # Used to convert the predicted state estimate into predicted sensor measurements at time k.
        # In this case, H will be the identity matrix since the estimated state maps directly to state measurements data
        # H has the same number of rows as sensor measurements and same number of columns as states.
        self.H=np.eye(5)

        
        # State model noise covariance matrix Q
        # When Q is large, the Kalman Filter tracks large changes in
        # the sensor measurements more closely than for smaller Q.
        # Q is a square matrix that has the same number of rows as states.
        self.Q=np.diag([1, 0.1, 0.01, 1.0, 1.0]) 
        

        # Sensor measurement noise covariance matrix R
        # Has the same number of rows and columns as sensor measurements.
        # If we are sure about the measurements, R will be near zero.
        self.coord_var = 0.01
        self.angle_var = 0.0001
        self.speed_var = 6.0
        self.omega_var = 0
        
        self.R=np.diag([self.coord_var, self.coord_var, self.angle_var, self.speed_var, self.omega_var])

        self.P=np.eye(5)
        
    def updateFk(self):
        '''
        Function that updates the state transition matrix Fk
        '''
        
        self.Fk=np.matrix([[1.0, 0, 0, self.dt*math.cos(self.x[2]), 0],
                           [0, 1.0,  0, self.dt*math.sin(self.x[2]), 0],
                           [0, 0, 1.0, 0, self.dt],
                           [0, 0, 0, 1.0, 0],
                           [0, 0, 0, 0, 1.0]])
        self.Fk = self.Fk.astype(float)

    def predict(self):
        # Update Fk
        self.updateFk()
        # Predict the state estimate (A Priori) at time k based on the state estimate at time k-1
        self.x=np.dot(self.Fk, self.x)
        # Predict the state covariance estimate based on the previous covariance and some noise
        self.P=np.dot(np.dot(self.Fk, self.P), self.Fk.T)+self.Q
        return self.x

    def update(self, z, CameraAccessible = True): # z[0:2] corrpesonds to measurement of camera, z[3:4] corresponds to measurements of wheels
        
        if CameraAccessible:
            self.H=np.eye(5)
            
            self.R=np.diag([self.coord_var, self.coord_var, self.angle_var, self.speed_var, self.omega_var])
        else: # CAUTION : IF CAMERAACCESSIBLE = FALSE, Z IS A 2x1 VECTOR (z[0:1] are now measrurements from wheels)
            # Measurement matrix H (now only v and omega can be observed)
            self.H=np.matrix([[0, 0, 0, 1, 0],
                              [0, 0, 0, 0, 1]])
            
            self.R=np.diag([self.speed_var, self.omega_var])
            
            
        # Compute Kalman gain
        S=np.dot(self.H, np.dot(self.P, self.H.T))+self.R
        inv_S = np.linalg.pinv(S.astype(float))
        K=np.dot(np.dot(self.P, self.H.T),inv_S)

        # Correction / innovation
        # Calculate an updated state estimate (A Posteriori) for time k
        self.x=self.x+np.dot(K, (z-np.dot(self.H, self.x)))
        # Update the state covariance estimate for time k
        I=np.eye(self.H.shape[1])
        self.P=(I-(K*self.H))*self.P


        return self.x

In [7]:
def dist(pt1, pt2):
    distance = ( (pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2 )**0.5
    return distance

In [8]:
def get_thymio_position(img, img_output):
    
    '''
    Function that outputs Thymio position, radius and orientation from camera frames using Aruco marker detection
    Parts of code taken from https://mecaruco2.readthedocs.io/en/latest/notebooks_rst/Aruco/aruco_basics.html

    Inputs: - img : image from camera in BGR format
            - img_output : copy of image img on which will be drawn Thymio's contours

    Output: - center : tuple (x,y) representing the center of the Thymio (in pixels)
            - radius : radius of the Thymio (in pixels)
            - angle : orientation of the Thymio (in radians)
            - thymio_detected : boolean set to True if Thymio was detected, False if not
    '''
    
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    parameters =  aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    frame_markers = aruco.drawDetectedMarkers(img_output, corners, ids)
    
    
    thymio_detected = False
    center = (0,0)
    radius = 0
    angle = 0
    
    if np.all(ids != None):
        for i in range(len(ids)):
            if ids[i] == 0:
                thymio_detected = True
                c = corners[i][0]

                top_center = (c[0:2, 0].mean(),c[0:2, 1].mean())
                bottom_center = (c[2:4, 0].mean(), c[2:4, 1].mean())
                left_center = (c[0:4:3, 0].mean(), c[0:4:3, 1].mean())
                right_center = (c[1:3, 0].mean(), c[1:3, 1].mean())
                points = np.array([ [c[2,0],c[2,1]], [c[3,0],c[3,1]], [left_center[0],left_center[1]], [right_center[0],right_center[1]] ])

                square_center = (int(c[:, 0].mean()), int(c[:, 1].mean()))
                radius = int(max( dist(square_center,c[0]), dist(square_center,c[1]), dist(square_center,c[2]), dist(square_center,c[3])))
                angle = math.atan2(top_center[1]-bottom_center[1],top_center[0]-bottom_center[0])
                
                
                center = (int(points[:, 0].mean()), int(points[:, 1].mean()))
                cv2.arrowedLine(img_output, (center[0],center[1]), (int(top_center[0]),int(top_center[1])), (255,255,0), 10)
                cv2.circle(img_output,square_center,radius,(255,0,0),10)

                break

        
    return center, radius, angle, thymio_detected

In [19]:
'''
Testing Kalman Filter
'''

global center, radius, angle, thymio_detected

forward_speed = 0
angular_speed = 0

KF=KalmanFilter(0.033, [500, 300, math.pi/4, forward_speed, angular_speed])



### INITIALIZE CAMERA
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
fps = int(cap.get(5))
print("fps:", fps)

success, img = cap.read()
imgResult = img.copy()
    
thymioLoc, thymioRadius, thymioAngle, thymioDetected = get_thymio_position(img, imgResult)


path = [[thymioLoc[0],thymioLoc[1]],[thymioLoc[0]+300,thymioLoc[1]+300],[thymioLoc[0]+500,thymioLoc[1]-300]]

### MAIN LOOP
count = 0

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

path_step = 0
    
# running infinite while loop so that
# program keep running until we close it
while True:
    success, img = cap.read()
    imgResult = img.copy()
    
    thymioLoc, thymioRadius, thymioAngle, thymioDetected = get_thymio_position(img, imgResult)
    prox = get_var("prox.horizontal")[0]
    #print(prox)
    if any([x>21000 and x < 31000 for x in prox]):
            print("obstacle is very close") 
            avoid = False
            #prox =prox
            var_prox= np.array(prox)
            if (var_prox[:1].sum() > var_prox[3:4].sum()) and (var_prox[2] < 3000):
                set_var(motor_left_target= 100)
                set_var(motor_right_target= -100)
                print("should turn left")
            elif var_prox[2] > 3000: 
                set_var(motor_left_target= 100)
                set_var(motor_right_target= -100)
                print("should turn left")

            else:
                print("should turn right")
                set_var(motor_left_target= -100)
                set_var(motor_right_target= 100)


    elif any([x>3700 for x in prox]):
        print("obstacle is close")
        # ON PLACE ROTATION
        avoid = False
        #prox =prox
        var_prox= np.array(prox)
        if (var_prox[:1].sum() > var_prox[3:4].sum()):
            #node.send_set_variables(move(0, -100))
            set_var(motor_left_target= 0)
            set_var(motor_right_target= -100)
            print("motor right")
        elif (var_prox[:1].sum() < var_prox[3:4].sum()):
            set_var(motor_left_target= -100)
            set_var(motor_right_target= 0)
            print("motor left")
        else:
            set_var(motor_left_target= -100)
            set_var(motor_right_target= 0)

    else:
        # here global path planning	
        set_var(motor_left_target=100)
        set_var(motor_right_target = 100)
        wait = 1 
        # set motors normally here
        time.sleep(wait)
        avoid = True
    

    # KALMAN -------------------------------------------------------------------
    KF.predict()
    if thymioDetected:
        z = np.array([thymioLoc[0],thymioLoc[1],thymioAngle, forward_speed, angular_speed])
        z = np.expand_dims(z, axis=-1)
    else:
        z = np.array([forward_speed, angular_speed]) 
        z = np.expand_dims(z, axis=-1)
    KF.update(z,thymioDetected)
    cv2.circle(imgResult, (int(KF.x[0]), int(KF.x[1])), 10, (255,0,255), 10)
    # KALMAN ----------------------------------------------------------------------
    count = count + 1
    #print(count)
    #print('Thymio center:',thymioLoc, '  Angle:',thymioAngle, '  Thymio Detected:' ,thymioDetected)
    #print('Kalman center:',KF.x[0:2],'  Kalman angle:',KF.x[2], 'Kalman speed and omega', KF.x[3:5])
    
    #Path tracking ----------------------------------------------------------------
    x = KF.x.item(0)
    y = KF.x.item(1)
    angle = KF.x.item(2)
    
    #if Point(x,y).distance(Point())
    
    traj_seg = Segment(Point(*tuple(path[path_step])),Point(*tuple(path[path_step+1])))
    traj_ray = Ray(Point(*tuple(path[path_step+1])),Point(*tuple(path[path_step])))
    robot_dot = Point(x,y)
    robot_ray = Ray(Point(*tuple(path[path_step+1])),robot_dot)
    vert_ray = Ray(Point(*tuple(path[path_step])),angle=0)
    origin = Point(0,0)
    error = traj_seg.distance(robot_dot)
    imgResult = cv2.putText(imgResult, "Error: {0}".format(int(error)), (200,200), cv2.FONT_HERSHEY_SIMPLEX ,2, (0,0,255),2, cv2.LINE_AA)
    ref_angle = np.arctan((path[path_step][1]-path[path_step+1][1])/(path[path_step][0]-path[path_step+1][0])) #vert_ray.angle_between(traj_ray)
    if robot_ray.closing_angle(traj_ray) < 0:
        error = -error

    
    kp = 0.02
    correction = kp*error
    correction = max(-np.pi/2, correction)
    correction = min(np.pi/2, correction)
    target_angle = ref_angle + correction
    
    speed_offset = 50
    
    kp_angle = 100
    angle_error = target_angle-angle
    speed_diff = kp_angle*angle_error
    set_var(motor_left_target = int(speed_offset + speed_diff))
    set_var(motor_right_target = int(speed_offset - speed_diff))

    
    imgResult = cv2.putText(imgResult, "Angle: {0}".format(int(angle*180/np.pi)), (200,300), cv2.FONT_HERSHEY_SIMPLEX ,2, (0,0,255),2, cv2.LINE_AA)
    imgResult = cv2.putText(imgResult, "ref angle: {0}".format(int(ref_angle*180/np.pi)), (200,400), cv2.FONT_HERSHEY_SIMPLEX ,2, (0,0,255),2, cv2.LINE_AA)
    imgResult = cv2.putText(imgResult, "correction: {0}".format(int(correction*180/np.pi)), (200,500), cv2.FONT_HERSHEY_SIMPLEX ,2, (0,0,255),2, cv2.LINE_AA)
    imgResult = cv2.putText(imgResult, "target_angle: {0}".format(int(target_angle*180/np.pi)), (200,600), cv2.FONT_HERSHEY_SIMPLEX ,2, (0,0,255),2, cv2.LINE_AA)
    imgResult = cv2.putText(imgResult, "speed_diff: {0}".format(int(speed_diff)), (200,700), cv2.FONT_HERSHEY_SIMPLEX ,2, (0,0,255),2, cv2.LINE_AA)
    imgResult = cv2.putText(imgResult, "angle_error: {0}".format(int(angle_error*180/np.pi)), (200,800), cv2.FONT_HERSHEY_SIMPLEX ,2, (0,0,255),2, cv2.LINE_AA)
    imgResult = cv2.putText(imgResult, "angle traj: {0}".format(int(robot_ray.closing_angle(traj_ray)*180/np.pi)), (200,900), cv2.FONT_HERSHEY_SIMPLEX ,2, (0,0,255),2, cv2.LINE_AA)

    # displaying output on Screen
    imgResult = cv2.line(imgResult,tuple(path[0]),tuple(path[1]),(255,255,255),1)
    imgResult = cv2.line(imgResult,tuple(path[1]),tuple(path[2]),(255,255,255),1)
    cv2.imshow("Result", imgResult)
    
    c = cv2.waitKey(1)
    if c == 27:
        cv2.destroyWindow("Result")
        break

Unchecked Raise(exc=Call(func=Name(id='IOError', ctx=Load()), args=[Constant(value='Cannot open webcam', kind=None)], keywords=[]), cause=None)
fps: 30


TypeError: set_var() takes 0 positional arguments but 2 were given

In [10]:
set_var(motor_left_target = 0)
set_var(motor_right_target = 0)